<a href="https://colab.research.google.com/github/ddddd-ai/CP/blob/main/%E5%8A%A0%E7%8F%AD%E6%A0%B8%E6%B4%BE%E5%96%AE-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/你的帳號/你的repo/blob/main/你的筆記本.ipynb)

SyntaxError: invalid syntax (<ipython-input-1-e550826d1a72>, line 1)

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from docx import Document
from docx.oxml.ns import qn
from docx.shared import Pt

# ======== 設定 ========
打卡檔案 = "打卡.xlsx"
word範本 = "加班單範本.docx"
輸出資料夾 = "加班核派單"
os.makedirs(輸出資料夾, exist_ok=True)

# ======== 工具函數 ========
def fmt(dt):
    return f"{dt.year} 年 {dt.month} 月 {dt.day} 日 {dt.hour} 時 {dt.minute} 分"

def 時數轉共計文字(時數):
    h = int(時數)
    m = int((時數 - h) * 60)
    return f"共計{h}時{m}分"

# ======== 讀取資料並分析加班 ========
df = pd.read_excel(打卡檔案)
df["刷卡時間"] = pd.to_datetime(df["刷卡時間"])
df["日期"] = df["刷卡時間"].dt.date
df = df[df["成功/失敗"].str.contains("成功", na=False)]

加班紀錄 = []

for 日期, group in df.groupby("日期"):
    打卡時間 = sorted(group["刷卡時間"])
    if len(打卡時間) < 2:
        continue

    上班 = 打卡時間[0]
    下班 = 打卡時間[-1]

    正常下班 = 上班 + timedelta(hours=8, minutes=30)
    加班起 = 正常下班 + timedelta(minutes=30)

    if 下班 <= 加班起:
        continue

    # 實際加班時間（分鐘）
    加班分鐘 = (下班 - 加班起).total_seconds() / 60
    有效分鐘 = int(加班分鐘 // 30) * 30
    if 有效分鐘 < 30:
        continue

    申報時數 = 有效分鐘 / 60
    共計文字 = 時數轉共計文字(申報時數)

    加班紀錄.append({
        "date": 日期,
        "ot_start_time": 加班起,
        "ot_end_time": 下班,
        "申報時數": 申報時數,
        "共計文字": 共計文字
    })

summary_df = pd.DataFrame(加班紀錄)

# ======== 產出 Word ========
for _, row in summary_df.iterrows():
    doc = Document(word範本)

    日期 = row["date"]
    預計起 = datetime.combine(日期, datetime.strptime("17:00", "%H:%M").time())
    預計迄 = 預計起 + timedelta(hours=row["申報時數"])
    預計共計 = 時數轉共計文字(row["申報時數"])

    實際起 = pd.to_datetime(row["ot_start_time"])
    實際迄 = pd.to_datetime(row["ot_end_time"])
    實際共計 = 時數轉共計文字(row["申報時數"])  # ✅ 改由申報時數換算

    填表內容 = {
        (2, 1): fmt(預計起),
        (2, 6): fmt(預計迄),
        (2, 9): 預計共計,
        (3, 1): fmt(實際起),
        (3, 6): fmt(實際迄),
        (3, 9): 實際共計,
    }

    for (r, c), val in 填表內容.items():
        cell = doc.tables[0].rows[r].cells[c]
        cell.text = val
        run = cell.paragraphs[0].runs[0]
        run.font.name = 'DFKai-SB'
        run._element.rPr.rFonts.set(qn('w:eastAsia'), 'DFKai-SB')
        run.font.size = Pt(10)

    # 填單日期
    填單日期文字 = f"填單日期：{日期.year}年{日期.month}月{日期.day}日"
    for para in doc.paragraphs:
        if "填單日期" in para.text:
            para.text = 填單日期文字
            run = para.runs[0]
            run.font.name = 'DFKai-SB'
            run._element.rPr.rFonts.set(qn('w:eastAsia'), 'DFKai-SB')
            run.font.size = Pt(10)
            break

    檔名 = f"加班單_{日期}.docx"
    doc.save(os.path.join(輸出資料夾, 檔名))

print("✅ 所有加班核派單已產出完成！")
